# TP 6 : types construits

## Gestion des overflow

On considère le type suivant :

In [3]:
type entier = Infini | MoinsInfini | Entier of int

type entier = Infini | MoinsInfini | Entier of int


On rappelle que `max_int` est le plus grand entier représentable en OCaml et `min_int` le plus petit :

In [ ]:
max_int;;
min_int;;
max_int + 1;;  (* un dépassement du plus grand entier donne le plus petit *)

On voudrait pouvoir additionner deux `entier` $a$ et $b$ en prennant en compte les dépassements de `int` (integer overflow), et en donnant `Infini` dans ce cas.  
1. Pourquoi ne peut-on pas tout simplement tester si $a + b >$ `max_int` ?   
2. Pourquoi peut-on tester à la place si `max_int` $ - ~a \leq b$ avec $a \geq 0$ ?
3. Quel test similaire pourrait-on utiliser pour savoir si $a + b <$ `min_int` ?
4. Écrire une fonction `add_int : int -> int -> entier` telle que `add_int` ajoute 2 `int`en tenant compte des dépassements (on renvoie `Infini` s'il y a un dépassement de `max_int` et `MoinsInfini` s'il y a un dépassement de `min_int` par le bas).  
5. Écrire une fonction `add : entier -> entier -> entier` ajoutant 2 entiers en tenant compte des dépassements.  
On pourra écrire `failwith "Indeterminé"` dans le cas d'une forme indéterminé.
6. Écrire une fonction `oppose : entier -> entier` renvoyant l'opposé d'un entier. Par exemple, `oppose Infini` renvoie `MoinsInfini`, `oppose (Entier a)` renvoie `Entier (-a)...`
7. Écrire une fonction `sub` effectuant la soustraction.
8. Pour savoir si $a \times b$ fait un dépassement, on propose la façon suivante :
- Calculer $c = a \times b$
- Regarder si $c/a$ est égal à $b$  
Pourquoi cela fonctionne ? À quelle autre erreur faut-il faire attention dans cette méthode ?
9. Écrire une fonction `mult : entier -> entier -> entier` multipliant 2 entiers en tenant compte des dépassements

1. Par définition, aucun entier ne peut être supérieur à `max_int`.
2. Déjà, $a + b >$ `max_int` implique que $a \geq 0$ (car $b$ est un entier dans est inférieur à `max_int`). De plus `max_int` $- a$ n'effectue pas de dépassement. Donc le test fonctionne sans risque de dépassement.
3. $a + b < $`min_int` se réécrit en $a < 0$ et `min_int`$ - a > b$.  

In [ ]:
(*4*)
let add int a b = match a, b with
 |( a>b || min_int -b< a) -> Infini
 |(a<b || min_int -b<a) -> MoinsInfini
 | (a+b);;

In [14]:
(*5*)
let add a b = match a b with 
    |Entier(a), Entier(b) when max_int - a <= b-> Infini
    |Entier (a), Entier (b) when min_int + a >=b -> MoinsInfini
    |Infini, Entier (_)|Entier (_), Infini-> Infini
    |MoinsInfini,Entier (_) | Entier (_), MoinsInfini -> MoinsInfini
    |Entier (a1), Entier (b1) -> Entier (a1 + b1)
    |_-> failwith "indetermine"

val add : ('a -> entier * entier) -> 'a -> entier = <fun>


In [4]:
(*6*)
let oppose a = match a with
    |infini -> MoinsInfini
    |moinsinfini -> Infini
    |entier (a1) when a1= min_int -> Infini
    |entier (a1) -> Entier (-a1)
    

In [ ]:
let rec mult a b = match a b with
    |Infini, Infini | Infini, MoinsInfini -> MoinsInfini
    |MoinsInfini, Infini | Infini, MoinsInfini -> MoinsInfini
    |MoinsInfini, Entier (e) | Entier (e), MoinsInfiniwhen e>0 -> MoinsInfini
    |MoinsInfini, Entier (e)| Entier (e), MoinsInfini when e<0 ->  Infini
    |Infini, Entier (a) | Entier (a), Infini when e<0 -> MoinsInfini
    |Infini, Entier (e) | Entier (e), Infini when e>0 ->Infini
    |Entier (a), Entier (b) -> if a = 0 then Entier (0)
                                else let c = a*b in
                                if c/a=b then Entier (a*b)
                                else mult (mult(Infini) (Entier (a)) (Entier (b));
    | _->fail

## Tableaux dynamiques

Il est impossible d'ajouter ou supprimer un élément dans tableau (`array`) en OCaml.  
Les tableaux dynamiques (ou : redimensionnables) permettent d'ajouter un élément `e`, en recréant un tableau plus grand dans lequel on recopie tous les éléments ainsi que `e`.  
Voici la définition de tableau dynamique que nous allons utiliser :

In [ ]:
type 'a array_dyn = {mutable t : 'a array; mutable n : int};;

`n` indique le nombre de cases du tableau `t` que l'on considère comme faisant partie du tableau dynamique. Les indices au delà de `n` dans `t` sont ignorés.  
À chaque fois que l'on voudra ajouter un élément `e` à un tableau dynamique `d` :
- s'il reste de la place dans `d.t` (c'est à dire si `d.n < Array.length t`), on met `e` dans `e` dans `d.t.(n)` et on met à jour `d.n`
- sinon, on créé un nouveau tableau `t'` de taille `2n`, on recopie `d.t` dedans, on ajoute `e` puis on remplace `d.t` par `t'.`

1. Écrire une fonction `add` ajoutant un élément dans un `array_dyn`. 
2. Quelle est la complexité dans le pire cas de `add` ?
3. On suppose que l'on ajoute $n$ éléments (avec `add` à un tableau dynamique de taille initiale 1. Montrer que la complexité totale de ces $n$ opérations est O($n$) (autrement dit, la complexité moyenne ou **complexité amortie** d'une opération est O(1))

In [9]:
let rec add t e = 
    let t = ref d.t.(0)  in
    let t' = ref 2*d.t.(n)
        for i = 0 to Array.length t-1 do
    if t = [||] then [|e|]
    else if d.n < Array.length t then dn := !dn + e
    else Array.copy t then add e 
    done;
    !t';;
    
 
   
    

error: compile_error